# 聚类


## 载入相应模块

In [1]:
import re
import json
from sklearn.cluster import KMeans,MiniBatchKMeans,AffinityPropagation,MeanShift,SpectralClustering,AgglomerativeClustering,DBSCAN
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer as ss
import jieba
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from sklearn.mixture import GaussianMixture

In [2]:
stp = stopwords.words('english')
stm = ss('english')

## 处理tweet文件 构造text和label的list

In [3]:
d=open('Tweets2.txt','r+')
text=[]
label=[]
texts=d.readlines()
for i in texts:
    text.append(json.loads(i)["text"])
    label.append(json.loads(i)["cluster"])
print(text[:10])
print(min(label))

['brain fluid buildup delay giffords rehab', 'trailer talk week movie rite mechanic week opportunity', 'rnc appoints chairman tampa convention effort visit tampa republican nati tampa fl', 'gbagbo camp futile cut ivory coast economy', 'chinese president lost translation powerful leader meet expect tran', 'england fishing community current management system broken edf', 'protest reform start yemen hundred anti government protester gathered sanaa', 'stuxnet lead chernobyl russian', 'iphone share smartphones phone', 'uploaded youtube video nba final lakers celtic game memory highlig']
1


In [4]:
text2=[]

for i in text:
    s=i.split()
    print(s)
    z=[]
    for word in s:
        word = stm.stem(word)
        z.append(word)

    text2.append(z)
print(text2[1000:1002])



['brain', 'fluid', 'buildup', 'delay', 'giffords', 'rehab']
['trailer', 'talk', 'week', 'movie', 'rite', 'mechanic', 'week', 'opportunity']
['rnc', 'appoints', 'chairman', 'tampa', 'convention', 'effort', 'visit', 'tampa', 'republican', 'nati', 'tampa', 'fl']
['gbagbo', 'camp', 'futile', 'cut', 'ivory', 'coast', 'economy']
['chinese', 'president', 'lost', 'translation', 'powerful', 'leader', 'meet', 'expect', 'tran']
['england', 'fishing', 'community', 'current', 'management', 'system', 'broken', 'edf']
['protest', 'reform', 'start', 'yemen', 'hundred', 'anti', 'government', 'protester', 'gathered', 'sanaa']
['stuxnet', 'lead', 'chernobyl', 'russian']
['iphone', 'share', 'smartphones', 'phone']
['uploaded', 'youtube', 'video', 'nba', 'final', 'lakers', 'celtic', 'game', 'memory', 'highlig']
['feed', 'epic', 'sci', 'fi', 'car', 'ad', 'kia', 'optimum', 'super', 'bowl', 'commercial', 'video', 'trendhunter', 'supe']
['naughty', 'facebook', 'started', 'sharing', 'personal', 'detail', 'site'

['student', 'juvenile', 'arthritis', 'apply', 'college', 'scholarship', 'staff', 'report', 'arthritis', 'foundation']
['love', 'ncis', 'arc', 'outcome', 'funeral']
['leon', 'payn', 'bottega', 'veneta', 'coming']
['egypt', 'protest', 'gauntlet', 'officer', 'beating', 'stick']
['yemen', 'president', 'quit', 'february', 'updated']
['suicide', 'bomber', 'kill', 'moscow', 'airport', 'explosion', 'ripped', 'interna', 'lukewilliamss']
['starbucks', 'trenta', 'cup', 'hold', 'entire', 'bottle', 'wine', 'random', 'starbucks', 'trenta', 'college']
['vincent', 'tabak', 'man', 'accused', 'jo', 'yeates', 'murder', 'deeply', 'saddened', 'death', 'telegraph']
['florida', 'judge', 'declares', 'entire', 'healthcare', 'law', 'void', 'month', 'half', 'ago', 'virginia', 'judge', 'key', 'pr']
['motorola', 'xoom', 'super', 'bowl', 'commercial', 'tip', 'hat', 'apple', 'spot', 'video']
['fly', 'fishing', 'magazine', 'guide']
['cost', 'financial', 'aid', 'chicago', 'school', 'professional', 'psychology']
['emin

## 得到文本的tf-idf矩阵

In [5]:
vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值

In [6]:
tfidf=transformer.fit_transform(vectorizer.fit_transform(text)) #第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
weight=tfidf.toarray()   #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重 
vocab=vectorizer.get_feature_names()#获取词袋模型中的所有词
print(vocab[:10])

['aacons', 'aaron', 'aaup', 'aba', 'abby', 'abc', 'abdallah', 'abdullah', 'abel', 'absence']


In [7]:
weight.shape

(2472, 5097)

## 尝试各聚类方法

### KMeans

In [9]:
mykms=KMeans(n_clusters=max(label))
y1=mykms.fit_predict(weight)

In [10]:
mykms2=MiniBatchKMeans(n_clusters=max(label))
y2=mykms2.fit_predict(weight)

###  AffinityPropagation

In [11]:
mykms3=AffinityPropagation()
y3=mykms3.fit_predict(weight)

### MeanShift

In [10]:
mykms4=MeanShift(bandwidth=0.4,min_bin_freq=2)
y4=mykms4.fit_predict(weight)

### SpectralClustering

In [17]:
mykms5=SpectralClustering(n_clusters=max(label))
y5=mykms5.fit_predict(weight)

### Ward hierarchical clustering

In [14]:
mykms6=AgglomerativeClustering(n_clusters=max(label))
y6=mykms6.fit_predict(weight)

### AgglomerativeClustering

In [15]:
mykms7=AgglomerativeClustering(n_clusters=max(label),linkage ='average')
y7=mykms7.fit_predict(weight)

### DBSCAN

In [16]:
mykms8=DBSCAN(eps=0.7, min_samples=1)
y8=mykms8.fit_predict(weight)

### GaussianMixture 

In [23]:
mykms9=GaussianMixture(n_components=11)
y9=mykms9.fit_predict(weight)

## NMI评价（按上述方法顺序依次排列）

In [64]:
NMI(label,y1)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.8025938826711199

In [71]:
NMI(label,y2)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.6607457455387594

In [77]:
NMI(label,y3)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.7831643850554082

In [12]:
NMI(label,y4)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.6994805035385545

In [18]:
NMI(label,y5)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.680920890243362

In [89]:
NMI(label,y6)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.77587403569932

In [94]:
NMI(label,y7)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.9004539868135747

In [121]:
NMI(label,y8)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.706878988031472

In [24]:
NMI(label,y9)

G:\anaconda3\envs\python35\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.613721759544673